In [14]:
from nltk import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import textwrap
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer,PorterStemmer

In [15]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dellb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dellb\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
df=pd.read_csv('datasets/bbc_text_cls.csv')

In [3]:
df.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [18]:
doc=df[df['labels']=='business']['text'].sample(random_state=42)

In [21]:
def wrap(x):
    return textwrap.fill(x,replace_whitespace=False,fix_sentence_endings=True)

In [22]:
print(wrap(doc.iloc[0]))

Christmas sales worst since 1981

UK retail sales fell in December,
failing to meet expectations and making it by some counts the worst
Christmas since 1981.

Retail sales dropped by 1% on the month in
December, after a 0.6% rise in November, the Office for National
Statistics (ONS) said.  The ONS revised the annual 2004 rate of growth
down from the 5.9% estimated in November to 3.2%. A number of
retailers have already reported poor figures for December.  Clothing
retailers and non-specialist stores were the worst hit with only
internet retailers showing any significant growth, according to the
ONS.

The last time retailers endured a tougher Christmas was 23 years
previously, when sales plunged 1.7%.

The ONS echoed an earlier
caution from Bank of England governor Mervyn King not to read too much
into the poor December figures.  Some analysts put a positive gloss on
the figures, pointing out that the non-seasonally-adjusted figures
showed a performance comparable with 2003. The Novembe

In [23]:
sents=sent_tokenize(doc.iloc[0].split("\n",1)[1])

In [24]:
featurizer=TfidfVectorizer(stop_words=stopwords.words('english'),norm='l1')#no bias towards longer sentences

In [25]:
X=featurizer.fit_transform(sents)

In [27]:
def get_sentence_score(tfidf_row):
    #avg of non-zero values
    x=tfidf_row[tfidf_row!=0]
    return x.mean()

In [28]:
scores=np.zeros(len(sents))
for i in range(len(sents)):
    score=get_sentence_score(X[i,:])
    scores[i]=score

In [29]:
sort_idx=np.argsort(-scores)

In [32]:
print("generated summary: ")
for i in sort_idx[:5]:
    print(wrap("%.2f %s" % (scores[i], sents[i])))

generated summary: 
0.14 A number of retailers have already reported poor figures for
December.
0.13 However, reports from some High Street retailers highlight the
weakness of the sector.
0.12 The ONS revised the annual 2004 rate of growth down from the 5.9%
estimated in November to 3.2%.
0.10 "Our view is the Bank of England will keep its powder dry and
wait to see the big picture."
0.10 And a British Retail Consortium survey found that Christmas 2004
was the worst for 10 years.


In [33]:
##funtion for abpve code

def summarize(text):
    #tokenize into sentences
    sents=sent_tokenize(text)
    
    #perform tf-idf
    X=featurizer.fit_transform(sents)
    
    #calculate score
    scores=np.zeros(len(sents))
    for i in range(len(sents)):
        score=get_sentence_score(X[i,:])
        scores[i]=score
    
    sort_idx=np.argsort(-scores)
    
    print("generated summary: ")
    for i in sort_idx[:5]:
        print(wrap("%.2f %s" % (scores[i], sents[i])))

## using textrank

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
#compute cosine similarity
S=cosine_similarity(X)

In [39]:
S.shape

(17, 17)

In [40]:
len(sents)

17

In [42]:
#normalize similarity matrix
S/=S.sum(axis=1, keepdims=True)

In [44]:
S[0].sum()

1.0

In [45]:
#Uniform transition matrix
U=np.ones_like(S)/len(S)

In [46]:
U[0].sum()

1.0

In [47]:
#smoothed similarity matrix
factor=0.15
S=(1-factor)*S+factor*U

In [48]:
S[0].sum()

1.0

In [49]:
#find the limiting/stationary distribution
eigenvals, eigenvecs=np.linalg.eig(S.T)

In [50]:
eigenvals

array([1.        , 0.24245466, 0.72108199, 0.67644122, 0.34790129,
       0.34417302, 0.3866884 , 0.40333562, 0.41608572, 0.44238593,
       0.63909999, 0.62556792, 0.58922572, 0.57452382, 0.48511399,
       0.51329157, 0.52975372])

In [51]:
eigenvecs[:,0]

array([-0.24206557, -0.27051337, -0.2213806 , -0.28613638, -0.25065894,
       -0.2499217 , -0.279622  , -0.21515455, -0.2226665 , -0.22745415,
       -0.2059112 , -0.20959727, -0.23526242, -0.24203809, -0.23663025,
       -0.2940483 , -0.20865607])

In [52]:
eigenvecs[:,0].dot(S)

array([-0.24206557, -0.27051337, -0.2213806 , -0.28613638, -0.25065894,
       -0.2499217 , -0.279622  , -0.21515455, -0.2226665 , -0.22745415,
       -0.2059112 , -0.20959727, -0.23526242, -0.24203809, -0.23663025,
       -0.2940483 , -0.20865607])

In [53]:
eigenvecs[:,0]/eigenvecs[:,0].sum()

array([0.05907327, 0.06601563, 0.05402535, 0.06982824, 0.06117038,
       0.06099047, 0.06823848, 0.05250595, 0.05433915, 0.05550753,
       0.05025022, 0.05114976, 0.05741304, 0.05906657, 0.05774684,
       0.07175905, 0.05092007])

In [55]:
limiting_dist=np.ones(len(S))/len(S)
threshold=1e-8
delta=float('inf')
iters=0
while delta>threshold:
    iters+=1
    
    #markov transition
    p=limiting_dist.dot(S)
    
    #compute change in limiting distribution
    delta=np.abs(p-limiting_dist).sum()
    
    #update limitng distribution
    limiting_dist=p
    
print(iters)

41


In [56]:
limiting_dist

array([0.05907327, 0.06601563, 0.05402534, 0.06982824, 0.06117038,
       0.06099047, 0.06823848, 0.05250595, 0.05433915, 0.05550753,
       0.05025022, 0.05114977, 0.05741304, 0.05906657, 0.05774685,
       0.07175905, 0.05092008])

In [57]:
limiting_dist.sum()

0.9999999999999982

In [58]:
np.abs(eigenvecs[:,0]/eigenvecs[:,0].sum()-limiting_dist).sum()

1.996473881354932e-08

In [59]:
scores=limiting_dist

In [60]:
sort_idx=np.argsort(-scores)

In [61]:
print("generated summary: ")
for i in sort_idx[:5]:
    print(wrap("%.2f %s" % (scores[i], sents[i])))

generated summary: 
0.07 "The retail sales figures are very weak, but as Bank of England
governor Mervyn King indicated last night, you don't really get an
accurate impression of Christmas trading until about Easter," said Mr
Shaw.
0.07 A number of retailers have already reported poor figures for
December.
0.07 The ONS echoed an earlier caution from Bank of England governor
Mervyn King not to read too much into the poor December figures.
0.07 Retail sales dropped by 1% on the month in December, after a 0.6%
rise in November, the Office for National Statistics (ONS) said.
0.06 Clothing retailers and non-specialist stores were the worst hit
with only internet retailers showing any significant growth, according
to the ONS.


## using libraries

In [62]:
!pip install sumy

     ---------------------------------------- 97.3/97.3 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     --------------------------------------- 10.1/10.1 MB 20.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for chardet from https://files.pythonhosted.org/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 199.4/199.4 kB ? eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21741 sha256=f49db59fcd1051b29574cdb8288bb69154

In [63]:
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

In [64]:
summarizer=TextRankSummarizer()
parser=PlaintextParser.from_string(
        doc.iloc[0].split("\n",1)[1],
            Tokenizer("english"))
summary=summarizer(parser.document,sentences_count=5)

In [65]:
summary

(<Sentence: Retail sales dropped by 1% on the month in December, after a 0.6% rise in November, the Office for National Statistics (ONS) said.>,
 <Sentence: The ONS revised the annual 2004 rate of growth down from the 5.9% estimated in November to 3.2%.>,
 <Sentence: The ONS echoed an earlier caution from Bank of England governor Mervyn King not to read too much into the poor December figures.>,
 <Sentence: Some analysts put a positive gloss on the figures, pointing out that the non-seasonally-adjusted figures showed a performance comparable with 2003.>,
 <Sentence: The November-December jump last year was roughly comparable with recent averages, although some way below the serious booms seen in the 1990s.>)

In [66]:
for s in summary:
    print(wrap(str(s)))

Retail sales dropped by 1% on the month in December, after a 0.6% rise
in November, the Office for National Statistics (ONS) said.
The ONS revised the annual 2004 rate of growth down from the 5.9%
estimated in November to 3.2%.
The ONS echoed an earlier caution from Bank of England governor Mervyn
King not to read too much into the poor December figures.
Some analysts put a positive gloss on the figures, pointing out that
the non-seasonally-adjusted figures showed a performance comparable
with 2003.
The November-December jump last year was roughly comparable with
recent averages, although some way below the serious booms seen in the
1990s.


In [67]:
summarizer=LsaSummarizer()
summary=summarizer(parser.document, sentences_count=5)
for s in summary:
    print(wrap(str(s)))

UK retail sales fell in December, failing to meet expectations and
making it by some counts the worst Christmas since 1981.
Morrisons, Woolworths, House of Fraser, Marks & Spencer and Big Food
all said that the festive period was disappointing.
And a British Retail Consortium survey found that Christmas 2004 was
the worst for 10 years.
Yet, other retailers - including HMV, Monsoon, Jessops, Body Shop and
Tesco - reported that festive sales were well up on last year.
Investec chief economist Philip Shaw said he did not expect the poor
retail figures to have any immediate effect on interest rates.


In [69]:
# from gensim.summarization import summarize
# summary=summarize(doc.iloc[0].split("\n",1)[1])
# print(wrap(summary))